# Casusopdracht - Films

**Klas:** V2B
**Studenten:** 
- Roan Gaasbeek
- Mathijs de Jong
- Luc Pikaar


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from astropy.cosmology import w0waCDM
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso

# 1. Business Understanding

## Doelstelling
Het doel van dit onderzoek is het vaststellen van de belangrijkste kenmerken (zoals budget
IMDb-scores, en sociale media populariteit) van films die goed presteren, om te begrijpen welke
factoren bijdragen aan succes in de filmindustrie.

## Onderzoeksvragen
De volgende onderzoeksvragen worden behandeld in deze opdracht:

1. In hoeverre is de bruto-omzet van een film te voorspellen op basis van de populariteit op **Facebook** en **IMDB**?
2. In hoeverre beïnvloedt het slagen voor de Bechdeltest de bruto-omzet van een film?
3. In hoeverre is het mogelijk om logische clusters te vinden o.b.v. `budget`, `omzet` en `IMDB-score`?

In [ ]:
# TODO - onderzoeksvraag 2:

# 2. Data understanding
De tweede fase in **CRISP-DM** is de **data understanding** dit houdt in:
- Data Collection
- Data Exploration & Analysis

## Data Collection

### Movies dataset
dit is de dataset die we uit deze opdracht hebben meegekregen

### Bechdeltest
De bechdel test is als volgt:
1. It has to have at least two (named) women in it
2. Who talk to each other
3. About something besides a man

https://bechdeltest.com/


In [ ]:
dataset = pd.read_csv('movie.csv')
bechdel_dataset = pd.read_csv('bechdel.csv')

## Data Exploration & Analysis
In deze fase wordt de dataset grondig onderzocht om patronen, relaties en potentieel problematische gegevens te identificeren. Het doel van data exploration is om een eerste inzicht te krijgen in de structuur van de data, trends te ontdekken en belangrijke kenmerken te identificeren die verder onderzocht moeten worden.

### Algemene verkenning
Om een eerste indruk van de dataset te krijgen, maken we gebruik van de `head`-functie. Deze functie geeft een overzicht van de eerste paar rijen van de dataset, waardoor we snel inzicht krijgen in de structuur van de data.

In [ ]:
dataset.head()

Om snel een overzicht te krijgen van alle kolommen en mogelijke features in de dataset, maken we gebruik van de `columns`-functie. Hiermee kunnen we eenvoudig de namen van de kolommen bekijken, wat nuttig is voor het bepalen van welke variabelen relevant zijn voor verdere analyse.

In [ ]:
dataset.columns

Vervolgens gebruiken we de `dtypes`-functie om de datatypen van elke kolom in de dataset te achterhalen. Dit helpt ons te begrijpen welke soort gegevens in elke kolom aanwezig zijn

In [ ]:
dataset.dtypes

We gebruiken de `describe`-functie om een overzicht te krijgen van de belangrijkste statistieken van de numerieke kolommen in de dataset. Deze functie geeft ons inzicht in de gemiddelde waarden, de spreiding, en de extremen van de data, waardoor we beter begrijpen hoe de gegevens zijn verdeeld.

In [ ]:
dataset.describe()

Voor een samenvatting van de niet-numerieke gegevens specificeren we in de `describe`-functie dat we ook objecttypen willen opnemen door `include='object'` toe te voegen.

In [ ]:
dataset.describe(include='object')

### Variabelen & Meetniveaus

In deze analyse zijn de volgende variabelen overwogen met hun verwachte impact op de bruto-omzet (gross):

- Director name & Actor names: De bekendheid van de regisseur en de hoofdrolspelers kan invloed hebben op de aantrekkingskracht van de film en daarmee de bruto-omzet.
- Facebook Likes: De sociale media populariteit van de regisseur en de acteurs kan de zichtbaarheid van de film vergroten en zo invloed op de bruto-omzet hebben.
- Budget: Een hoger budget kan leiden tot hogere bruto-omzet.
- IMDB score: Een hogere score op IMDB wijst vaak op betere filmkwaliteit en kan leiden tot meer kijkers en daarmee hogere bruto-omzet.


| Feature                 | Afhankelijk/Onafhankelijk | Meetniveau |
|:------------------------|:--------------------------|:-----------|
| Director Name           | Onafhankelijk             | Nominaal   |
| Actor 1 Name            | Onafhankelijk             | Nominaal   |
| Actor 2 Name            | Onafhankelijk             | Nominaal   |
| Actor 3 Name            | Onafhankelijk             | Nominaal   |
| Director Facebook Likes | Onafhankelijk             | Discreet   |
| Actor 1 Facebook Likes  | Onafhankelijk             | Discreet   |
| Actor 2 Facebook Likes  | Onafhankelijk             | Discreet   |
| Actor 3 Facebook Likes  | Onafhankelijk             | Discreet   |
| Movie Facebook Likes    | Onafhankelijk             | Discreet   |
| Budget                  | Onafhankelijk             | Continu    |
| IMDB Score              | Onafhankelijk             | Discreet   |
| Gross                   | Afhankelijk               | Continu    |

In [ ]:
kolommen = ['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'director_facebook_likes', 'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 'cast_total_facebook_likes','movie_facebook_likes', 'budget', 'imdb_score', 'gross']
data_analyse = dataset[kolommen]
data_analyse.head()


In [ ]:
data_analyse.describe()

### Target
De targetvariabele in dit onderzoek is de Bruto-omzet `gross`. Deze variabele vertegenwoordigt de inkomsten van een film.

Om inzicht te krijgen in de verspreiding van de bruto-omzetwaarden, zullen er twee histogrammen worden gebruikt. De eerste is zonder limiet hierin is het mogelijk om ook de uitschieters te zien. De tweede is ingezoomd op de waarde tot max 100 Miljoen. Dit helpt om de verdeling van de omzet in de dataset te visualiseren en te identificeren of er sprake is van een normale verdeling, uitschieters of andere patronen die van invloed kunnen zijn op de analyses.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
gross = data_analyse['gross'].dropna()
sns.histplot(gross / 10**6, bins=50, ax=axes[0])
axes[0].set_title('Bruto-omzetverdeling')
axes[0].set_xlabel('Bruto-omzet (Miljoenen)')
axes[0].set_ylabel('Frequentie')

sns.histplot(dataset['gross'] / 10**6, bins=200, ax=axes[1])
axes[1].set_title('Bruto-omzetverdeling met limiet (Max 100 miljoen)')
axes[1].set_xlabel('Bruto-omzet (Miljoenen)')
axes[1].set_ylabel('Frequentie')

# Zet de x-as limiet tot maximaal 200 miljoen
axes[1].set_xlim(0, 100)

plt.tight_layout()
plt.show()

Uit de histogrammen blijkt dat een groot deel van de `gross` waarden zich rond nul bevindt. Dit wijst mogelijk op een aanzienlijk aantal ontbrekende of ongeldige waarden in de dataset. Het kan zijn dat voor veel films de bruto-omzet ontbreekt. Dit zou de betrouwbaarheid van verdere analyses kunnen beïnvloeden.

In [ ]:
omzet = data_analyse['gross'].dropna()
omzet_log = np.log10(omzet + 1)

# Seaborn histogram plotten
plt.figure(figsize=(8, 6))
sns.histplot(omzet_log, bins=50, color='skyblue', edgecolor='black')

# Titel en labels toevoegen
plt.title('Bruto-omzet van films')
plt.xlabel('Bruto-omzet')
plt.ylabel('Aantal films')
plt.xticks(ticks=range(10), labels=["1","10","100","1k","10k","100k","1M", "10M", "100M", "1B"])

plt.grid(True)
plt.show()

Er is gekozen om een log10-transformatie toe te passen op de Bruot-omzet in plaats van een standaard histogram. Dit komt doordat er in de dataset veel films zijn met een bruto-omzet van 0 of zeer lage waarden. Door deze waarden op een lineaire schaal weer te geven, zouden de meeste films in een enkele categorie vallen, wat een onduidelijke visualisatie oplevert met weinig inzicht in de spreiding van de bruto-omzetten. De logaritmische schaal comprimeert de hoge uitschieters en spreidt de lagere waarden beter uit, waardoor de onderlinge verschillen beter zichtbaar worden en de visualisatie meer informatieve waarde krijgt.

### Features

#### Facebook likes
Om inzicht te krijgen in de populariteit van de verschillende actoren en de regisseur, hebben we de verdeling van de Facebook-likes per persoon afzonderlijk gevisualiseerd in de vorm van histogrammen. Deze grafieken tonen de frequentie van het aantal likes van de `director`, `acteur 1`, `acteur 2`, `acteur 3`, `cast_total` en de `movie`.

In [ ]:
dataset['director_facebook_likes'].describe()

In [ ]:
_, axes = plt.subplots(3, 2, figsize=(12, 10))

# Logaritmische transformatie van de 'director_facebook_likes' om uitschieters te comprimeren
director_likes = dataset['director_facebook_likes'].dropna()
director_likes_log = np.log10(director_likes + 1)
sns.histplot(director_likes_log, bins=20, kde=False, ax=axes[0, 0])

axes[0, 0].set_title('Regisseur FB Likes')
axes[0, 0].set_xlabel('Aantal Likes')
axes[0, 0].set_ylabel('Frequentie')
axes[0, 0].set_xticks([0, 1, 2, 3, 4, 5])
axes[0, 0].set_xticklabels(["1","10","100","1k","10k","100k"])

# Logaritmische transformatie van de 'actor_1_facebook_likes' om uitschieters te comprimeren
actor_1_likes = dataset['actor_1_facebook_likes'].dropna()
actor_1_likes_log = np.log10(actor_1_likes + 1)
sns.histplot(actor_1_likes_log, bins=20, kde=False, ax=axes[0, 1])

axes[0, 1].set_title('Acteur 1 FB Likes')
axes[0, 1].set_xlabel('Aantal Likes')
axes[0, 1].set_ylabel('Frequentie')
axes[0, 1].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[0, 1].set_xticklabels(["1","10","100","1k","10k","100k","1M"])

# Logaritmische transformatie van de 'actor_2_facebook_likes' om uitschieters te comprimeren
actor_2_likes = dataset['actor_2_facebook_likes'].dropna()
actor_2_likes_log = np.log10(actor_2_likes + 1)
sns.histplot(actor_2_likes_log, bins=20, kde=False, ax=axes[1, 0])

axes[1, 0].set_title('Acteur 2 FB Likes')
axes[1, 0].set_xlabel('Aantal Likes')
axes[1, 0].set_ylabel('Frequentie')
axes[1, 0].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[1, 0].set_xticklabels(["1","10","100","1k","10k","100k","1M"])

# Logaritmische transformatie van de 'actor_3_facebook_likes' om uitschieters te comprimeren
actor_3_likes = dataset['actor_3_facebook_likes'].dropna()
actor_3_likes_log = np.log10(actor_3_likes + 1)
sns.histplot(actor_3_likes_log, bins=20, kde=False, ax=axes[1, 1])

axes[1, 1].set_title('Acteur 3 FB Likes')
axes[1, 1].set_xlabel('Aantal Likes')
axes[1, 1].set_ylabel('Frequentie')
axes[1, 1].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[1, 1].set_xticklabels(["1","10","100","1k","10k","100k","1M"])

# Logaritmische transformatie van de 'cast_total_facebook_likes' om uitschieters te comprimeren
cast_total_likes = dataset['cast_total_facebook_likes'].dropna()
cast_total_likes_log = np.log10(cast_total_likes + 1)
sns.histplot(cast_total_likes_log, bins=20, kde=False, ax=axes[2, 0])

axes[2, 0].set_title('Cast Total FB Likes')
axes[2, 0].set_xlabel('Aantal Likes')
axes[2, 0].set_ylabel('Frequentie')
axes[2, 0].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[2, 0].set_xticklabels(["1", "10", "100", "1k", "10k", "100k", "1M"])

# Logaritmische transformatie van de 'movie_facebook_likes' om uitschieters te comprimeren
movie_likes = dataset['movie_facebook_likes'].dropna()
movie_likes_log = np.log10(movie_likes + 1)
sns.histplot(movie_likes_log, bins=20, kde=False, ax=axes[2, 1])

axes[2, 1].set_title('Movie Facebook Likes')
axes[2, 1].set_xlabel('Aantal Likes')
axes[2, 1].set_ylabel('Frequentie')
axes[2, 1].set_xticks([0, 1, 2, 3, 4, 5, 6])
axes[2, 1].set_xticklabels(["1", "10", "100", "1k", "10k", "100k", "1M"])

plt.tight_layout()
plt.show()

#### IMDB-Score

In [ ]:
# Veronderstel dat de dataset al geladen is en IMDb-scores bevat
imdb_scores = data_analyse['imdb_score'].dropna()

# Maak een boxplot voor de IMDb-scores
plt.figure(figsize=(8, 6))
sns.boxplot(x=imdb_scores)

# Instellen van de titel en labels
plt.title('IMDB Scores')
plt.ylabel('Cijfer IMDB')

# Weergeven van de plot
plt.grid(True)
plt.show()


#### Budget

In [ ]:
budget = data_analyse['budget'].dropna()
budget_log = np.log10(budget + 1)

# Seaborn histogram plotten
plt.figure(figsize=(8, 6))
sns.histplot(budget_log, bins=50, color='skyblue', edgecolor='black')

# Titel en labels toevoegen
plt.title('Budget')
plt.xlabel('Budget')
plt.ylabel('Aantal films')
plt.xticks(ticks=range(11), labels=["1","10","100","1k","10k","100k","1M", "10M", "100M", "1B", "10B"])

plt.grid(True)
plt.show()

## Data Kwaliteit

### Missende Waarden
Met de functie `isnull` kunnen we controleren hoeveel missende waarden we hebben. Hier voegen we nog een check aan toe om te kijken hoevel waarden 0 zijn

In [ ]:
(data_analyse.isnull() | (data_analyse == 0)).sum()


### Duplicate data
Met de functie `duplicated` checken we of er duplicates in de data zijn. Dit checken we door de waarde van kolommen `movie_title` en `movie_imdb_link` mee te geven aangezien dit eigenlijk de unieke waarden zouden moeten zijn.

In [ ]:
dataset[dataset.duplicated(['movie_title', 'movie_imdb_link'])]

### Uitschieters

In [ ]:
def bepaal_uitschieters(kolom):

        # Q1, Q3 en IQR bepalen
        q1 = data_analyse[kolom].quantile(0.25)
        q3 = data_analyse[kolom].quantile(0.75)
        iqr = q3 - q1

        # Limieten uitschieters berekenen
        klein_laag = q1 - 1.5 * iqr
        klein_hoog = q3 + 1.5 * iqr
        extreem_laag = q1 - 3 * iqr
        extreem_hoog = q3 + 3 * iqr

        print(f"Kolom: {kolom}")
        print(F"Totaal aantal waarden: {data_analyse[kolom].count()}")
        # Kleine uitschieters (1.5 - 3 IQR)
        kleine_uitschieters = data_analyse[kolom][((data_analyse[kolom] < klein_laag) & (data_analyse[kolom] > extreem_laag)) | ((data_analyse[kolom] > klein_hoog) & (data_analyse[kolom] < extreem_hoog))]
        print(f"Kleine uitschieters: {kleine_uitschieters.count()} ({kleine_uitschieters.count() / data_analyse[kolom].count() * 100:.2f}%)")
    
        # Extreme uitschieters (3+ IQR)
        extreme_uitschieters = data_analyse[kolom][(data_analyse[kolom] < extreem_laag) | (data_analyse[kolom] > extreem_hoog)]
        print(f"Extreme uitschieters: {extreme_uitschieters.count()} ({extreme_uitschieters.count() / data_analyse[kolom].count() * 100:.2f}%)")

In [ ]:
bepaal_uitschieters('director_facebook_likes')

In [ ]:
bepaal_uitschieters('actor_1_facebook_likes')

In [ ]:
bepaal_uitschieters('actor_2_facebook_likes')

In [ ]:
bepaal_uitschieters('actor_3_facebook_likes')

In [ ]:
bepaal_uitschieters('cast_total_facebook_likes')

In [ ]:
bepaal_uitschieters('movie_facebook_likes')

In [ ]:
bepaal_uitschieters('budget')

In [ ]:
bepaal_uitschieters('gross')

## Waarnemingen

- movie_facebook_likes heeft 930 extreme uitschieters, dat is behoorlijk veel. Zoals eerder benoemd, komt dit waarschijnlijk doordat er veel films zijn zonder facebook pagina, wat geregistreerd wordt als 0 likes.

- imdb_score heeft 0 extreme uitschieters. Dit lijkt ons logisch omdat de imdb score voor een film altijd tussen de 1 en 10 ligt. Er zal dus nooit een waarde van bijvoorbeeld 100 moeten zijn, wat een extreme uitschieter zou zijn.

- De kolommen movie_facebook_likes en imdb_score hebben beide 5043 waarden, maar budget en gross hebben minder waarden (4551 en 4159 waarden).  
We zullen hier een oplossing voor moeten vinden, door bijvoorbeeld de rijen met missende waarden te droppen van de dataset.

- Q1 voor movie_facebook_likes is 0, dit betekent dat ten minste 25% van de films 0 facebook likes hebben. Dit komt waarschijnlijk omdat deze films geen facebook-pagina hebben.

# 3. Data preparation


## Data Cleaning

In [ ]:
#TODO : titels hebben nog een chasing spatie

### Verwerken van de IMDb-link
In deze stap verwijderen we de volledige IMDb-link omdat we deze niet nodig hebben voor onze analyse. We extraheren echter de `imdb_id` uit de link, die we later kunnen gebruiken om de dataset te koppelen aan een externe dataset.


In [ ]:
dataset['movie_imdb_link'] = dataset['movie_imdb_link'].str.slice(start=28, stop=-17)

### kolomnamen
In deze stap wijzigen we de kolomnamen van de dataset naar het Nederlands voor een betere leesbaarheid en consistentie. Dit helpt om de data-analyse en rapportage te vergemakkelijken.


In [ ]:
dataset.rename(columns={
    'director_name': 'directeur',
    'director_facebook_likes': 'directeur_likes',
    'actor_1_name': 'acteur_1',
    'actor_1_facebook_likes': 'acteur_1_likes',
    'actor_2_name': 'acteur_2',
    'actor_2_facebook_likes': 'acteur_2_likes',
    'actor_3_name': 'acteur_3',
    'actor_3_facebook_likes': 'acteur_3_likes',
    'cast_total_facebook_likes': 'cast_totaal_likes',
    'movie_facebook_likes': 'film_likes',
    'movie_title': 'titel',
    'movie_imdb_link': 'imdb_id',
    'gross': 'omzet',
    'country': 'land',
    'budget': 'budget',
    'imdb_score': 'imdb_score',
}, inplace=True)

### Drop NaN
In deze stap controleren we op ontbrekende waarden in de kolommen die essentieel zijn voor onze analyses. We hebben ervoor gekozen om rijen met ontbrekende waarden te verwijderen in plaats van ze op te vullen, omdat het invullen van ontbrekende gegevens kan leiden tot vertekening van de analyses.


In [ ]:
columns_to_check = [
    'directeur',
    'acteur_1',
    'acteur_2',
    'acteur_3',
    'directeur_likes',
    'acteur_1_likes',
    'acteur_2_likes',
    'acteur_3_likes',
    'budget',
    'omzet'
]

# Controleer op ontbrekende waarden in de specifieke kolommen
missing_values = dataset[columns_to_check].isna().sum()

# Print de resultaten
print("Ontbrekende waarden per kolom:\n", missing_values)

dataset = dataset.dropna(subset=columns_to_check)

### Drop Duplicates 
In deze stap controleren we de dataset op duplicaten. Duplicaten kunnen ontstaan door foutieve gegevensinvoer of tijdens het samenvoegen van meerdere datasets.

In deze code controleren we specifiek op duplicaten op basis van de kolom `titel`, omdat we ervan uitgaan dat elke film een unieke titel moet hebben.
 
Na het identificeren van de duplicaten, verwijderen we deze om ervoor te zorgen dat onze dataset alleen unieke films bevat.


In [ ]:
# Controleert het aantal duplicates in de dataset.
totaal_duplicates = dataset.duplicated(['titel']).sum()
print(f"Aantal duplicates in de dataset: {totaal_duplicates}")

# Verwijder duplicates gebaseerd op de 'titel' kolom
dataset = dataset.drop_duplicates(['titel'])

# Controleer opnieuw of er nog duplicates zijn
remaining_duplicates = dataset.duplicated(['titel']).sum()
print(f"Aantal duplicates na verwijdering: {remaining_duplicates}")

### Local Currency to USD

In [ ]:
# Lees het CSV-bestand met de wisselkoersen
land_valuta_koers = pd.read_csv('land_valuta_koers.csv')

# Maak een dictionary voor de wisselkoersen
valuta_naar_usd_dict = pd.Series(land_valuta_koers['Koers_naar_USD'].values, index=land_valuta_koers['Land']).to_dict()

# Pas de omzet aan op basis van het land en de wisselkoers en rond af op 2 decimalen
dataset['omzet'] = dataset.apply(
    lambda row: round(row['omzet'] * valuta_naar_usd_dict.get(row['land'], 1), 2),  # Gebruik 1 als standaardwaarde
    axis=1
)

# Pas het budget aan op basis van het land en de wisselkoers en rond af op 2 decimalen
dataset['budget'] = dataset.apply(
    lambda row: round(row['budget'] * valuta_naar_usd_dict.get(row['land'], 1), 2),  # Gebruik 1 als standaardwaarde
    axis=1
)


# de eerste paar rijen om te controleren of de omzet goed is aangepast
dataset[['land', 'budget', 'omzet']].head()

### Kolomvolgorde
Het aanpassen van de volgorde van de kolommen in de dataset voor betere leesbaarheid en overzichtelijkheid.


In [ ]:
dataset = dataset[
    [
        'titel',              # title
        'directeur',          # director
        'directeur_likes',    # director_facebook_likes
        'acteur_1',           # actor_1
        'acteur_1_likes',     # actor_1_facebook_likes
        'acteur_2',           # actor_2
        'acteur_2_likes',     # actor_2_facebook_likes
        'acteur_3',           # actor_3
        'acteur_3_likes',     # actor_3_facebook_likes
        'cast_totaal_likes',   # cast_total_facebook_likes
        'film_likes',        # movie_facebook_likes
        'imdb_score',         # imdb_score
        'budget',             # budget
        'omzet'               # gross
    ]]

In [ ]:
dataset.head(10)

### Exporteren Dataset

In [ ]:
# Exporteer de dataset naar een CSV-bestand
dataset.to_csv('films.csv', index=False)

print("De dataset is succesvol geëxporteerd naar films.csv.")

## Correlation Analysis

### Facebook Likes

In [ ]:
# TODO: beschrijving in markdown erboven
dataset['totaal_likes'] = dataset['cast_totaal_likes'] + dataset['film_likes']
plt.figure(figsize=(8, 8))
plt.scatter(dataset['totaal_likes'], dataset['omzet'])
plt.title('Scatterplot van totale likes vs omzet')
plt.xlabel('Totaal aantal likes')
plt.ylabel('Omzet (USD)')
plt.grid(True)
plt.show()
correlatie = np.corrcoef(dataset['totaal_likes'], dataset['omzet'])[0,1]
print(f"De correlatie is {correlatie}")

### budget


In [ ]:
# TODO: beschrijving in markdown erboven
plt.figure(figsize=(8, 8))

data = dataset[['budget', 'omzet']].dropna() #TODO: dit moet nog in cleaning opgelost worden

plt.scatter(data['budget'], data['omzet'])
plt.title('Scatterplot van budget vs omzet')
plt.xlabel('Budget')
plt.ylabel('Omzet (USD)')
plt.grid(True)
plt.show()
correlatie = np.corrcoef(data['budget'], data['omzet'])[0,1]
print(f"De correlatie is {correlatie}")

### IMDB Score

In [ ]:
# TODO: beschrijving in markdown erboven
plt.figure(figsize=(8, 8))
plt.scatter(dataset['imdb_score'], dataset['omzet'])
plt.title('Scatterplot van imdb_score vs omzet')
plt.xlabel('IMDB score')
plt.ylabel('Omzet (USD)')
plt.grid(True)
plt.show()
correlatie = np.corrcoef(dataset['imdb_score'], dataset['omzet'])[0,1]
print(f"De correlatie is {correlatie}")

### Acteur 

Door films te herstructureren naar individuele film-acteur combinaties, kun je `acteur`-specifieke invloed op het succes van de film onderzoeken. Je kunt films clusteren op basis van de Facebook-likes van individuele acteurs en de bijbehorende filmomzet, en vervolgens patronen analyseren zoals welke acteurs de meeste invloed hebben op commerciële successen.

In [ ]:
actor_dataset = dataset[['titel', 'acteur_1', 'acteur_2', 'acteur_3', 'acteur_1_likes', 'acteur_2_likes', 'acteur_3_likes', 'omzet']]

# Reshape the DataFrame
actor_data = pd.DataFrame({
    'titel': actor_dataset['titel'].values.repeat(3),
    'acteur': actor_dataset[['acteur_1', 'acteur_2', 'acteur_3']].values.flatten(),
    'likes': actor_dataset[['acteur_1_likes', 'acteur_2_likes', 'acteur_3_likes']].values.flatten(),
    'omzet': actor_dataset['omzet'].values.repeat(3)
}).dropna()

actor_data.head()

In [ ]:
idx = actor_data.groupby('titel')['likes'].idxmax()

# Selecteer alleen de rijen met de meeste likes per film
actor_data = actor_data.loc[idx].reset_index(drop=True)
actor_data.head()

In [ ]:
# Scatter plot: Likes vs Gross
plt.figure(figsize=(8, 6))

# Maak een nieuwe kolom met log10 van likes
actor_data['log_likes'] = np.log10(actor_data['likes'] + 1)  # +1 om log(0) te vermijden

# Scatterplot met log10 op de x-as
sns.scatterplot(x='log_likes', y='omzet', data=actor_data)

plt.title('Acteur Populariteit vs. Film omzet')
plt.xlabel('Acteur Facebook Likes')
plt.ylabel('Film omzet')
plt.grid(True)
plt.xticks(ticks=[0,1,2,3,4,5,6], labels=["1", "10", "100", "1k", "10k", "100k", "1M"])  # Labels voor x-as
plt.show()

correlatie = np.corrcoef(actor_data['log_likes'], actor_data['omzet'])[0,1]
print(f"De correlatie is {correlatie}")


## Train test split voor baseline en regressie model


In [ ]:
# Hier kiezen we onze features en het target. Daarnaast splitsen we de training en validatie.

features = dataset[['imdb_score', 'film_likes', 'directeur_likes', 
        'acteur_1_likes', 'acteur_2_likes', 
        'acteur_3_likes', 'cast_totaal_likes']]

target = dataset['omzet']

features_train, features_val, target_train, target_val = train_test_split(features, target, random_state=70, test_size=0.23)
print("Training set:", features_train.shape, target_train.shape)
print("Validatie set:", features_val.shape, target_val.shape)

### Baseline model

In [ ]:
# In het baseline model berekenen we het gemiddelde uit van het target. We gebruiken een baseline model om te vergelijken met het
# supervised learning model.

gemiddelde_baseline = target_train.mean()

baseline_voorspelling = [gemiddelde_baseline] * len(target_val)

baseline_r2 = r2_score(target_val, baseline_voorspelling)
score = baseline_r2 * 100
print(f"R2 score: {score:.2f}%")

Een score van -0.03% betekent dat het model slechter voorspelt dan het gemiddelde. Het baseline model is daarom geen goed model om de target variabele 'gross' te voorspellen. 

### Supervised learning model

In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)
pd.DataFrame(model.coef_, features.columns, columns = ['Coeff'])

target_pred = model.predict(features_val)
r2 = r2_score(target_val, target_pred)
score_percentage = r2 * 100
print(f"R2 score: {score_percentage:.2f}%")

Wat betekent een R² score van 29% precies?
Een R² van 0.29 betekent dat het model slechts 29% van de variatie in de afhankelijke variabele in dit geval omzet verklaart. Dit betekent dat 71% van de variatie niet wordt verklaard door het model en door andere factoren komt. 

R² = 1 (100%) betekent dat het model perfect past en alle variatie in de data verklaart.
R² = 0 (0%) betekent dat het model helemaal geen verklaring biedt voor de variatie in de data. Dit is even 'slecht' als het gemiddelde.

Zoals hier boven te zien zit het model er redelijk vaak naast en enkele keren in de buurt. De conclusie die wij hieruit trekken is dat dit model met een score van 29% niet betrouwbaar is om de omzet te voorspellen.

In [ ]:
plt.figure(figsize=(8,6))

plt.scatter(target_val, target_pred, edgecolor='k')

#Diagonale rode lijn die de perfecte voorspelling laat zien
plt.plot([target_val.min(), target_val.max()], [target_val.min(), target_val.max()], color='red', linestyle='--')

plt.title('Werkelijke waarde (omzet) vs voorspelde waarde (omzet)')
plt.xlabel('Werkelijke waarde (omzet)')
plt.ylabel('Voorspelde waarde (omzet)')
plt.grid(True)
plt.show()


De rode lijn in het plot is de perfect voorspelde waarde. In het plot kan je zien dat bij een lagere bruto inkomst het model beter werkt en naarmate de gross hoger wordt het steeds minder goed werkt.